## Importing packages

In [2]:
## Import packages

import os # something very general

import matplotlib.pyplot as plt     #for plotting
from matplotlib.cm import get_cmap

import cartopy.crs as crs    #for geography maps
from cartopy.feature import NaturalEarthFeature

from netCDF4 import Dataset   #netCDF4 library and wrf-python
from wrf import (getvar, ALL_TIMES)

import xarray as xr



### Importing data

In [17]:
## Define my own custom filepath, from where I should ingest wrf data
myPath   = r'/home/olddog/Documents/Python_Scipts/WRF_nesting_prediction_Web/dataTest'
filename = 'wrfout_d03_2022-05-06_01:00:00'

In [18]:
## Read in the file
ncfile = Dataset(os.path.join(myPath, filename))


varname = 'p'

# 
data = getvar(ncfile, varname, timeidx=ALL_TIMES)


### Exploring the content

In [19]:
## Print basic info
print('User requested "{}"'.format(varname))
print(data.dims, data.coords)

User requested "p"
('Time', 'bottom_top', 'south_north', 'west_east') Coordinates:
    XLONG    (south_north, west_east) float32 32.07 32.09 32.11 ... 34.7 34.73
    XLAT     (south_north, west_east) float32 34.51 34.51 34.51 ... 35.85 35.84
    XTIME    (Time) float32 5.478e+04 5.484e+04 5.49e+04 ... 5.61e+04 5.616e+04
  * Time     (Time) datetime64[ns] 2022-05-06T01:00:00 ... 2022-05-07


In [20]:
print("These are the metadata associated with variable ", varname)
data.attrs
    

These are the metadata associated with variable  p


{'FieldType': 104,
 'MemoryOrder': 'XYZ',
 'description': 'pressure',
 'units': 'Pa',
 'stagger': '',
 'coordinates': 'XLONG XLAT XTIME',
 'projection': LambertConformal(stand_lon=24.957000732421875, moad_cen_lat=35.07198715209961, truelat1=35.071998596191406, truelat2=35.071998596191406, pole_lat=90.0, pole_lon=0.0),
 '_FillValue': 1e+20,
 'missing_value': 1e+20}

### Exploring the global content of the WHOLE wrf file

In [21]:
nr_variables = len(ncfile.variables)
first_N_variables = 30

print("Filename {} contains {} variables.\n-->At the moment, the user exported only variable {}".format(filename, nr_variables, varname) )


ds = xr.open_dataset(os.path.join(myPath, filename))
c = 1
print('\nExtracting first {} variable names...'.format(first_N_variables))
for item in list(ds.variables)[0:first_N_variables]:
    print("{:2d}) {}".format(c, item))
    _temp = getvar(ncfile, varname, timeidx=ALL_TIMES)
    print(_temp.coords)   
    print("{} in {}".format(_temp.description,_temp.units ))
        
    print("")
    c=c+1

Filename wrfout_d03_2022-05-06_01:00:00 contains 396 variables.
-->At the moment, the user exported only variable p

Extracting first 30 variable names...
 1) Times
Coordinates:
    XLONG    (south_north, west_east) float32 32.07 32.09 32.11 ... 34.7 34.73
    XLAT     (south_north, west_east) float32 34.51 34.51 34.51 ... 35.85 35.84
    XTIME    (Time) float32 5.478e+04 5.484e+04 5.49e+04 ... 5.61e+04 5.616e+04
  * Time     (Time) datetime64[ns] 2022-05-06T01:00:00 ... 2022-05-07
pressure in Pa

 2) XLAT
Coordinates:
    XLONG    (south_north, west_east) float32 32.07 32.09 32.11 ... 34.7 34.73
    XLAT     (south_north, west_east) float32 34.51 34.51 34.51 ... 35.85 35.84
    XTIME    (Time) float32 5.478e+04 5.484e+04 5.49e+04 ... 5.61e+04 5.616e+04
  * Time     (Time) datetime64[ns] 2022-05-06T01:00:00 ... 2022-05-07
pressure in Pa

 3) XLONG
Coordinates:
    XLONG    (south_north, west_east) float32 32.07 32.09 32.11 ... 34.7 34.73
    XLAT     (south_north, west_east) float32 34

### Check what variables have vertical dimension

In [22]:
has_vert = []
for item in ds:
    if 'bottom_top' in ds[item].dims: 
        has_vert.append(item)
print("{} variables out of {} have a vertical dimension (counting only unstaggered)".format(len(has_vert), len(ds)))        

155 variables out of 389 have a vertical dimension (counting only unstaggered)


### Slicing

In [65]:
# WRF-python is built on xarray and, thus, accepts same syntax and commands: 
# Lets try slicing the data:

# Does the data have vertical dimension?? Let us code something that is flexible enough to understand it, and act accordingly

if "bottom_top" in data.dims:
    sliced = data.isel(Time=0, west_east = 20, south_north=25, bottom_top=1)
    print('Exctrating indices!')
    print('\tTime:0')
    print('\tx:20')
    print('\ty:25')
    print('\tz:1')
else:
    sliced = data.isel(Time=0, west_east = 20, south_north=25)
    print('Exctrating indices!')
    print('\tTime:0')
    print('\tx:20')
    print('\ty:25')
    print('\tVariable {} has NO vertical cooridnate'.format(varname))
print('**AFTER** the slicing, the dimensional rank was reduced and the original time-varying datacube is now... a point\nIf we print its description we have:\n')
print(sliced)


Exctrating indices!
	Time:0
	x:20
	y:25
	z:1
**AFTER** the slicing, the dimensional rank was reduced and the original time-varying datacube is now... a point
If we print its description we have:

<xarray.DataArray 'pressure' ()>
array(93810.984, dtype=float32)
Coordinates:
    XLONG    float32 32.55
    XLAT     float32 34.93
    XTIME    float32 5.478e+04
    Time     datetime64[ns] 2022-05-06T01:00:00
Attributes:
    FieldType:      104
    MemoryOrder:    XYZ
    description:    pressure
    units:          Pa
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LambertConformal(stand_lon=24.957000732421875, moad_cen_l...
    _FillValue:     1e+20
    missing_value:  1e+20


## Extracting Timeseries